# **Microplane MS1 - uncoupled N-T interface**

This sheets shows the application of the microplane integration
on an example of VUNTIM (vectorized implementation of the uncoupled NT Interface Model.

In [ ]:
%matplotlib widget
from bmcs_matmod.msx import MSX
import matplotlib.pylab as plt
import numpy as np
np.seterr(divide='ignore', invalid='ignore');

The example below demonstrate a case with ideally plastic response which delivers
several peaks in the response. At the same time, it can be used to demonstrate that 
$\nu \leq 0.25$ leads does not deliver any solution upon return mapping due to a square root overflow.

Once $E_\mathrm{T}$ becomes negative, the return mapping algorithm cannot work properly.
Is there some thermodynamically admissible interpretation of this case in the interface?
This can be posed in two questions:
 - is it posssible to obtain a positive dissipation of an interface even for negative
   value of $E_\mathrm{T}$?
 - if yes, is it allowed to reach an inelastic domain in the tangential direction?

In [ ]:
E = 37000
nu = 0.2
mp = dict(          
Ad = 800,
eps_0  = 0.00007,
sig_0  = 25,
K_N = 14000,
gamma_N = 8000,
sig_T_0  = 2.2,
K_T  = 1200,
gamma_T  = 65000,
S_T  = 0.0027,
c_T  = 8.7,
r_T  = 9.2,
a  = 0.001,
e_T  = 7.5)

In [ ]:
%matplotlib widget
mpl = MSX()
mpl.trait_set(E=E, nu=nu, 
          mic='untim', eps_max=0.01, n_eps=30, double_pvw=True)
mpl.mic_.trait_set(**mp)
mpl.mic_.eps_max = 0.02
mpl.interact()

In [ ]:
from ibvpy.api import XDomainSinglePoint, MATS3DScalarDamage, TStepBC, BCDof
xmodel = XDomainSinglePoint()
m = TStepBC(
    domains=[(xmodel, mpl),],
    bc=[BCDof(
            var='u', dof=0, value=-0.01,
        )]
)

In [ ]:
xmodel.map_U_to_field(np.array([0,0,0,0,0,1]))

In [ ]:
sdomain, msx = m.domains[0]
# mpl.get_corr_pred(**m.fe_domain[0].state_n)

In [ ]:
m.sim.tline.trait_set(step=0.001)
m.sim.reset()
m.sim.run()

In [ ]:
%matplotlib widget
import matplotlib.pylab as plt
_, (ax, ax_nu) = plt.subplots(1,2)
ax.plot(m.hist.U_t[:,0], m.hist.F_t[:,0])
ax.plot(m.hist.U_t[:,2], m.hist.F_t[:,0])
ax_nu.plot(m.hist.U_t[:,0], -m.hist.U_t[:,1] / m.hist.U_t[:,0])